In [1]:
import numpy as np
import sympy as sp
from sympy.diffgeom import Manifold, Patch, CoordSystem


In [92]:
dim = 32
# L = 1.0
# dx = sp.Rational(1, (dim + 1))
# u0 = 1.0
L, dx, c0 = sp.symbols('L dx c0', real=True)

udim = 1

m = Manifold('M', dim)
p = Patch('P', m)

coordstr = 'x0'
for d in range(1, dim):
    coordstr += ' x%d' % d
x = sp.symbols(coordstr, real=True)
vx = sp.Matrix(x)

u = []
for d in range(udim):
    u += [sp.symbols('u%d' % d, real=True)]

In [95]:
Dx = sp.banded(dim, {-1: - sp.Rational(1, 2) / dx, 1: sp.Rational(1, 2) / dx})
Dx[-1,-2] = - 1 / dx
Dx[-1,-1] = 1 / dx
# Dx = sp.banded(dim, {-1: - 0.5 / dx, 1: 0.5 / dx})
# Dx[-1,-2] = - 1.0 / dx
# Dx[-1,-1] = 1.0 / dx

f = c0 * Dx * vx
# f

In [96]:
g = sp.Matrix([[0] for k in range(dim)])
g[4] = 1
# g

In [97]:
def LieBracket(f_, g_, vx_):
    return g_.jacobian(vx_) * f_ - f_.jacobian(vx_) * g_

In [101]:
from copy import deepcopy

Cmat = sp.Matrix([])
tmp = deepcopy(g)
for d in range(dim):
    Cmat = Cmat.col_insert(d, tmp)
#     tmp = LieBracket(f, tmp, vx) * dx / c0
    tmp = LieBracket(f, tmp, vx) * dx / c0
    tmp = tmp / tmp.norm(sp.oo)

Cmat

Matrix([
[0,  0,   0,    0,  1/6,    0,  -1/4,    0,    2/7,        0,  -75/251,        0,  275/912,          0,    -77/257,          0,    364/1231,           0,     -39/134,            0,        75/262,             0,     -550/1953,              0,     374/1351,              0,     -323/1204,               0,       1235/4739,                0,       -665/2621,               0],
[0,  0,   0, -1/3,    0,  1/2,     0, -4/7,      0,    25/42,        0, -275/461,        0,     77/131,          0,   -364/633,           0,      78/139,           0,      -75/137,             0,      550/1029,             0,     -1496/2863,            0,        221/432,             0,      -1729/3447,               0,        3325/6751,               0,        -200/413],
[0,  0, 1/2,    0, -2/3,    0,   3/4,    0, -11/14,        0,  200/251,        0, -121/152,          0,    203/257,          0,   -962/1231,           0,     207/268,            0,      -100/131,             0,     1474/1953,              0,  

In [105]:
Crhs = sp.Matrix([[0] for k in range(dim)])
Crhs[-1] = 1
Csol = Cmat.transpose().solve(Crhs)
Csol /= Csol.norm(sp.oo)
Csol

Matrix([
[-1/2],
[ 3/4],
[  -1],
[   1],
[   0],
[   1],
[   1],
[ 3/4],
[ 1/2],
[   0],
[ 1/2],
[-3/4],
[   1],
[  -1],
[   0],
[  -1],
[  -1],
[-3/4],
[-1/2],
[   0],
[-1/2],
[ 3/4],
[  -1],
[   1],
[   0],
[   1],
[   1],
[ 3/4],
[ 1/2],
[   0],
[ 1/2],
[-3/8]])

In [108]:
def LieDerivative(f_, ld_, vx_):
    return f_.dot(sp.diff(ld_, vx_))

In [112]:
lds = Csol.dot(vx)
LD_lds = []
for k in range(dim):
    LD_lds += [LieDerivative(Cmat.col(k), lds, vx)]
print(LD_lds)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1/68316808]


In [124]:
xt, xt_coeffs = [], [1]
tmp = deepcopy(lds)
for k in range(dim):
    xt += [tmp]
    tmp = sp.simplify(LieDerivative(f, tmp, vx) * dx / c0)
    xt_coeffs += [sp.Matrix(sp.Poly(tmp).coeffs()).norm(sp.oo)]
    tmp = sp.simplify(tmp / xt_coeffs[-1])
    
vxt = sp.Matrix(xt)
vxt

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         -x0/2 + 3*x1/4 + x10/2 - 3*x11/4 + x12 - x13 - x15 - x16 - 3*x17/4 - x18/2 - x2 - x20/2 + 3*x21/4 - x22 + x23 + x25 + x26 + 3*x27/4 + x28/2 + x3 + x30/2 - 3*x31/8 + x5 + x6 + 3*x7/4 + x8/2],
[                                                                                                                                                                                                                                                                                     

In [132]:
pdiag = lambda d: xt_coeffs[d] * sp.Pow(c0 / dx, d)
Px = sp.banded(dim, {0: pdiag})

Ft = sp.banded(dim, {1: 1})
print(Px.shape)
print(Ft.shape)
Ftt = Px.inv() * (Ft * Px)

(32, 32)
(32, 32)
